In [ ]:
# Add Origination Date and first payment data to check if the AUC improves

In [204]:
pip install pyspark

In [397]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("CreditRisk") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
df = spark.read.csv("dataset.csv", inferSchema = True, header = True, sep=",")

In [398]:
#Check dimension's
print((df.count(),len(df.columns)))

(614, 13)


In [399]:
#Check for the schema
df.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [400]:
df.show(200)

+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0|    Graduate|           No|           5849|              0.0|      null|             360|             1|        Urban|          Y|
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|          Y

In [401]:

my_data=df.na.fill("unknown",["Gender"]) \
    .na.fill("unknown",["Married"])\
    .na.fill("unknown",["Education"])\
    .na.fill("unknown",["Self_Employed"])\
    .na.fill(0,["ApplicantIncome"])\
    .na.fill(0,["CoapplicantIncome"])\
    .na.fill(0,["LoanAmount"])\
    .na.fill(0,["Loan_Amount_Term"])\
    .na.fill(0,["Credit_History"])\
    .na.fill("unknown",["Property_Area"])\
    .na.fill("unknown",["Loan_Status"])

In [402]:
#Top 5 records
my_data.show(200)

+--------+-------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID| Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+-------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001002|   Male|     No|         0|    Graduate|           No|           5849|              0.0|         0|             360|             1|        Urban|          Y|
|LP001003|   Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001005|   Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|     

In [403]:
#Datatypes of the columns
my_data.dtypes

[('Loan_ID', 'string'),
 ('Gender', 'string'),
 ('Married', 'string'),
 ('Dependents', 'string'),
 ('Education', 'string'),
 ('Self_Employed', 'string'),
 ('ApplicantIncome', 'int'),
 ('CoapplicantIncome', 'double'),
 ('LoanAmount', 'int'),
 ('Loan_Amount_Term', 'int'),
 ('Credit_History', 'int'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string')]

In [404]:
#Drop unwanted columns
my_data = my_data.drop(*['Loan_ID','Self_Employed','Dependents'])
my_data.columns

['Gender',
 'Married',
 'Education',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History',
 'Property_Area',
 'Loan_Status']

In [405]:
# get the dimensions of the data
(my_data.count() , len(my_data.columns))

(614, 10)

In [406]:
# import sql function pyspark
import pyspark.sql.functions as f
# fill the null values
my_data = my_data.fillna(0)
# null values in each column
data_agg = my_data.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in my_data.columns])
data_agg.show()


+------+-------+---------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|Gender|Married|Education|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+------+-------+---------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|     0|      0|        0|              0|                0|         0|               0|             0|            0|          0|
+------+-------+---------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+



In [407]:
my_data.dtypes

[('Gender', 'string'),
 ('Married', 'string'),
 ('Education', 'string'),
 ('ApplicantIncome', 'int'),
 ('CoapplicantIncome', 'double'),
 ('LoanAmount', 'int'),
 ('Loan_Amount_Term', 'int'),
 ('Credit_History', 'int'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string')]

In [408]:

#Preprocessing steps
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# create object of StringIndexer class and specify input and output column
SI_Gender = StringIndexer(inputCol='Gender',outputCol='Gender_Index')
SI_Married = StringIndexer(inputCol='Married',outputCol='Married_Index')
SI_Education = StringIndexer(inputCol='Education',outputCol='Education_Index')
SI_ApplicantIncome = StringIndexer(inputCol='ApplicantIncome',outputCol='ApplicantIncome_Index')
SI_CoapplicantIncome = StringIndexer(inputCol='CoapplicantIncome',outputCol='CoapplicantIncome_Index')
SI_LoanAmount = StringIndexer(inputCol='LoanAmount',outputCol='LoanAmount_Index')
SI_Loan_Amount_Term = StringIndexer(inputCol='Loan_Amount_Term',outputCol='Loan_Amount_Term_Index')
SI_Credit_History = StringIndexer(inputCol='Credit_History',outputCol='Credit_History_Index')
SI_Property_Area = StringIndexer(inputCol='Property_Area',outputCol='Property_Area_Index')
SI_Loan_Status = StringIndexer(inputCol='Loan_Status',outputCol='Loan_Status_Index')




# transform the data
my_data = SI_Gender.fit(my_data).transform(my_data)
my_data = SI_Married.fit(my_data).transform(my_data)
my_data = SI_Education.fit(my_data).transform(my_data)
my_data = SI_ApplicantIncome.fit(my_data).transform(my_data)
my_data = SI_CoapplicantIncome.fit(my_data).transform(my_data)
my_data = SI_LoanAmount.fit(my_data).transform(my_data)
my_data = SI_Loan_Amount_Term.fit(my_data).transform(my_data)
my_data = SI_Credit_History.fit(my_data).transform(my_data)
my_data = SI_Property_Area.fit(my_data).transform(my_data)
my_data = SI_Loan_Status.fit(my_data).transform(my_data)


In [409]:
my_data.show(300)

+-------+-------+------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+------------+-------------+---------------+---------------------+-----------------------+----------------+----------------------+--------------------+-------------------+-----------------+
| Gender|Married|   Education|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|Gender_Index|Married_Index|Education_Index|ApplicantIncome_Index|CoapplicantIncome_Index|LoanAmount_Index|Loan_Amount_Term_Index|Credit_History_Index|Property_Area_Index|Loan_Status_Index|
+-------+-------+------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+------------+-------------+---------------+---------------------+-----------------------+----------------+----------------------+--------------------+-------------------+-----------------+
|   Male|     No|    Graduate|    

In [410]:
my_data.dtypes

[('Gender', 'string'),
 ('Married', 'string'),
 ('Education', 'string'),
 ('ApplicantIncome', 'int'),
 ('CoapplicantIncome', 'double'),
 ('LoanAmount', 'int'),
 ('Loan_Amount_Term', 'int'),
 ('Credit_History', 'int'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string'),
 ('Gender_Index', 'double'),
 ('Married_Index', 'double'),
 ('Education_Index', 'double'),
 ('ApplicantIncome_Index', 'double'),
 ('CoapplicantIncome_Index', 'double'),
 ('LoanAmount_Index', 'double'),
 ('Loan_Amount_Term_Index', 'double'),
 ('Credit_History_Index', 'double'),
 ('Property_Area_Index', 'double'),
 ('Loan_Status_Index', 'double')]

In [411]:
# view the transformed data
my_data.select('Gender', 'Gender_Index','Married', 'Married_Index','Education','Education_Index','ApplicantIncome','ApplicantIncome_Index','CoapplicantIncome','CoapplicantIncome_Index','LoanAmount','LoanAmount_Index','Loan_Amount_Term','Loan_Amount_Term_Index','Credit_History','Credit_History_Index','Property_Area','Property_Area_Index','Loan_Status','Loan_Status_Index',).show()

+------+------------+-------+-------------+------------+---------------+---------------+---------------------+-----------------+-----------------------+----------+----------------+----------------+----------------------+--------------+--------------------+-------------+-------------------+-----------+-----------------+
|Gender|Gender_Index|Married|Married_Index|   Education|Education_Index|ApplicantIncome|ApplicantIncome_Index|CoapplicantIncome|CoapplicantIncome_Index|LoanAmount|LoanAmount_Index|Loan_Amount_Term|Loan_Amount_Term_Index|Credit_History|Credit_History_Index|Property_Area|Property_Area_Index|Loan_Status|Loan_Status_Index|
+------+------------+-------+-------------+------------+---------------+---------------+---------------------+-----------------+-----------------------+----------+----------------+----------------+----------------------+--------------+--------------------+-------------+-------------------+-----------+-----------------+
|  Male|         0.0|     No|        

In [414]:
# create object and specify input and output column
OHE = OneHotEncoder(inputCols=['Gender_Index', 'Married_Index','Education_Index','ApplicantIncome_Index','CoapplicantIncome_Index','LoanAmount_Index','Loan_Amount_Term_Index','Credit_History_Index','Property_Area_Index','Loan_Status_Index'],outputCols=['Gender_OHE', 'Married_OHE','Education_OHE','ApplicantIncome_OHE','CoapplicantIncome_OHE','LoanAmount_OHE','Loan_Amount_Term_OHE','Credit_History_OHE','Property_Area_OHE','Loan_Status_OHE'])

# transform the data
my_data = OHE.fit(my_data).transform(my_data)

# view and transform the data
my_data.select('Gender', 'Gender_Index','Gender_OHE').show()

+------+------------+-------------+
|Gender|Gender_Index|   Gender_OHE|
+------+------------+-------------+
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|Female|         1.0|(2,[1],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
|  Male|         0.0|(2,[0],[1.0])|
+------+------------+-------------+
only showing top 20 rows



In [415]:
my_data.columns

['Gender',
 'Married',
 'Education',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History',
 'Property_Area',
 'Loan_Status',
 'Gender_Index',
 'Married_Index',
 'Education_Index',
 'ApplicantIncome_Index',
 'CoapplicantIncome_Index',
 'LoanAmount_Index',
 'Loan_Amount_Term_Index',
 'Credit_History_Index',
 'Property_Area_Index',
 'Loan_Status_Index',
 'Education_OHE',
 'Loan_Status_OHE',
 'CoapplicantIncome_OHE',
 'Gender_OHE',
 'LoanAmount_OHE',
 'Property_Area_OHE',
 'ApplicantIncome_OHE',
 'Loan_Amount_Term_OHE',
 'Credit_History_OHE',
 'Married_OHE']

In [416]:
from pyspark.ml.feature import VectorAssembler

# specify the input and output columns of the vector assembler
assembler = VectorAssembler(inputCols=['Gender_Index',
 'Married_Index',
 'Education_Index',
 'ApplicantIncome_Index',
 'CoapplicantIncome_Index',
 'LoanAmount_Index',
 'Loan_Amount_Term_Index',
 'Credit_History_Index',
 'Property_Area_Index',
 'Education_OHE',
 'CoapplicantIncome_OHE',
 'Gender_OHE',
 'LoanAmount_OHE',
 'Property_Area_OHE',
 'ApplicantIncome_OHE',
 'Loan_Amount_Term_OHE',
 'Credit_History_OHE',
 'Married_OHE'],
                           outputCol='features')

# fill the null values
my_data = my_data.fillna(0)

# transform the data
final_data = assembler.transform(my_data)

In [418]:
final_data.show(300)

+-------+-------+------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+------------+-------------+---------------+---------------------+-----------------------+----------------+----------------------+--------------------+-------------------+-----------------+-------------+---------------+---------------------+-------------+-----------------+-----------------+-------------------+--------------------+------------------+-------------+--------------------+
| Gender|Married|   Education|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|Gender_Index|Married_Index|Education_Index|ApplicantIncome_Index|CoapplicantIncome_Index|LoanAmount_Index|Loan_Amount_Term_Index|Credit_History_Index|Property_Area_Index|Loan_Status_Index|Education_OHE|Loan_Status_OHE|CoapplicantIncome_OHE|   Gender_OHE|   LoanAmount_OHE|Property_Area_OHE|ApplicantIncome_OHE|Loan_Amount_Term_OHE|Credit_History_OH

In [419]:
# view the transformed vector
final_data.select('features','Loan_Status_Index').show(25)

+--------------------+-----------------+
|            features|Loan_Status_Index|
+--------------------+-----------------+
|(1020,[1,3,8,9,10...|              0.0|
|(1020,[3,4,5,8,9,...|              1.0|
|(1020,[3,5,8,9,10...|              0.0|
|(1020,[2,3,4,5,8,...|              0.0|
|(1020,[1,3,5,8,9,...|              0.0|
|(1020,[3,4,5,8,9,...|              0.0|
|(1020,[2,3,4,5,8,...|              0.0|
|(1020,[3,4,5,7,9,...|              1.0|
|(1020,[3,4,5,8,9,...|              0.0|
|(1020,[3,4,5,9,59...|              1.0|
|(1020,[3,4,5,8,9,...|              0.0|
|(1020,[4,5,8,9,11...|              0.0|
|(1020,[3,4,5,8,9,...|              0.0|
|(1020,[1,3,4,5,8,...|              1.0|
|(1020,[3,4,5,6,8,...|              0.0|
|(1020,[1,3,5,8,9,...|              0.0|
|(1020,[1,2,3,5,6,...|              0.0|
|(1020,[0,1,3,5,7,...|              1.0|
|(1020,[2,3,5,8,10...|              1.0|
|(1020,[3,4,5,6,8,...|              0.0|
|(1020,[2,3,5,7,8,...|              1.0|
|(1020,[3,4,5,8,

In [420]:
#Model_Dataframe
model_df = final_data.select(['features','Loan_Status_Index'])
model_df = model_df.withColumnRenamed("Loan_Status_Index","label")
model_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)



In [421]:
model_df.show(100)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1020,[1,3,8,9,10...|  0.0|
|(1020,[3,4,5,8,9,...|  1.0|
|(1020,[3,5,8,9,10...|  0.0|
|(1020,[2,3,4,5,8,...|  0.0|
|(1020,[1,3,5,8,9,...|  0.0|
|(1020,[3,4,5,8,9,...|  0.0|
|(1020,[2,3,4,5,8,...|  0.0|
|(1020,[3,4,5,7,9,...|  1.0|
|(1020,[3,4,5,8,9,...|  0.0|
|(1020,[3,4,5,9,59...|  1.0|
|(1020,[3,4,5,8,9,...|  0.0|
|(1020,[4,5,8,9,11...|  0.0|
|(1020,[3,4,5,8,9,...|  0.0|
|(1020,[1,3,4,5,8,...|  1.0|
|(1020,[3,4,5,6,8,...|  0.0|
|(1020,[1,3,5,8,9,...|  0.0|
|(1020,[1,2,3,5,6,...|  0.0|
|(1020,[0,1,3,5,7,...|  1.0|
|(1020,[2,3,5,8,10...|  1.0|
|(1020,[3,4,5,6,8,...|  0.0|
|(1020,[2,3,5,7,8,...|  1.0|
|(1020,[3,4,5,8,9,...|  0.0|
|(1020,[2,3,4,5,7,...|  1.0|
|(1020,[0,2,3,4,5,...|  1.0|
|(1020,[3,4,5,7,9,...|  1.0|
|(1020,[3,5,9,10,2...|  0.0|
|(1020,[3,4,5,9,16...|  0.0|
|(1020,[2,3,4,5,8,...|  0.0|
|(1020,[1,2,3,5,8,...|  1.0|
|(1020,[0,1,3,4,5,...|  0.0|
|(1020,[3,4,5,7,8,...|  1.0|
|(1020,[1,3,5,

In [446]:
#Split into training & testing Dataframe
training_df,test_df = model_df.randomSplit([0.75,0.25])

In [447]:
training_df.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1020,[0,1,2,3,4,...|  0.0|
|(1020,[0,1,2,3,5,...|  1.0|
|(1020,[0,1,2,3,5,...|  1.0|
|(1020,[0,1,2,3,5,...|  0.0|
|(1020,[0,1,2,3,5,...|  1.0|
|(1020,[0,1,2,3,5,...|  0.0|
|(1020,[0,1,2,3,5,...|  0.0|
|(1020,[0,1,2,3,5,...|  0.0|
|(1020,[0,1,2,3,5,...|  0.0|
|(1020,[0,1,2,3,5,...|  0.0|
|(1020,[0,1,2,3,5,...|  0.0|
|(1020,[0,1,2,3,5,...|  1.0|
|(1020,[0,1,2,3,7,...|  1.0|
|(1020,[0,1,3,4,5,...|  1.0|
|(1020,[0,1,3,4,5,...|  1.0|
|(1020,[0,1,3,4,5,...|  0.0|
|(1020,[0,1,3,4,5,...|  1.0|
|(1020,[0,1,3,4,5,...|  0.0|
|(1020,[0,1,3,4,5,...|  0.0|
|(1020,[0,1,3,4,5,...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [449]:
#Create a logistic regression model object
from pyspark.ml.classification import LogisticRegression
log_reg=LogisticRegression().fit(training_df)

In [467]:
lr_predictions = log_reg.transform(test_df)
lr_predictions.select("prediction","label","features").show(5)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
lr_evaluator = BinaryClassificationEvaluator()
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))


+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(1020,[0,1,2,3,5,...|
|       1.0|  1.0|(1020,[0,1,3,4,5,...|
|       1.0|  0.0|(1020,[0,1,3,4,5,...|
|       1.0|  1.0|(1020,[0,1,3,4,5,...|
|       0.0|  0.0|(1020,[0,1,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.650433


In [468]:
lr_summary=log_reg.summary

In [469]:
#Overall accuracy of the classification model
lr_summary.accuracy

1.0

In [470]:
lr_summary.areaUnderROC

1.0

In [445]:
#Precision of both classes
print(lr_summary.precisionByLabel)

[1.0, 1.0]


In [453]:
predictions = log_reg.transform(test_df)


In [454]:
predictions.select('label','prediction').show(50)


+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|     